In [ ]:
import pandas as pd
import json
import requests
import urllib.request
import numpy as np
import os
import time

## import all the summaries

In [ ]:
path = '../data/20210315_sample/'

# Slaat de bestandsnamen en summaries op als lists
name_list = []
sum_list = []

for f in os.listdir(path):
    if f.endswith('.txt'):
        name_list.append(f[:-4])
        sum_list.append(open(path+f, 'r').read())

In [ ]:
# manier om meer te laten zien van een dataframe item.
pd.set_option('display.max_colwidth', 1000)

# Store in dataframe, totale dataset beter in np array dit weglaten?
df = pd.DataFrame()
df['filename'] = name_list
df['summary'] = sum_list
# df.head()

In [ ]:

# ts = df[df['filename']== '61FB7787-C915-44EA-B7B2-0047A99ABF12']['summary']
# print("Samenvatting voor de text in volgende cell:")
# print(ts.to_string(index=False))

## import the transcribed text

##  the rigth timeframe to acces the target text to summarize
### 1. store the start and end time with the corresponding summaries

In [ ]:
def min_to_ms(timestamp):
    # input timestamp: 01:mm:ss
    # output ms
    ts_split = timestamp.split(':')
    ts_split = list(map(int, ts_split))
    ms = (ts_split[1]*60 + ts_split[2]) *1000
    return ms

In [ ]:
# convert start-/endtime to ms[]
df_mani = pd.read_csv(path+'manifest.csv', delimiter =';')
df_mani.columns = ['filename','date', 'starttime', 'endtime', 'jsonurl', 'mediafile']
df_mani['starttime'] = df_mani['starttime'].apply(min_to_ms)
df_mani['endtime'] = df_mani['endtime'].apply(min_to_ms)
# df_mani.head()

In [ ]:
# append to df
df_time = pd.merge(df, df_mani, on=['filename'])
df_time.head()

### 2. retrieve corresponding transcript based on timestamps

In [ ]:
def get_target_text(jsonurl, starttime, endtime):

#     # open de json file
#     with urllib.request.urlopen(jsonurl) as url:
#         data_json = json.loads(url.read().decode())
#         print(url)
    data_json = json.loads(requests.get(jsonurl).text)
        
    trans_text = ""
    for part in data_json:
        for i in range(len(part['result'])):
            if (starttime <= int(part['result'][i][1])) & (int(part['result'][i][2]) <= endtime +1000):
                trans_text += part['result'][i][0] + " "
    return trans_text


In [ ]:
# retrieve the transcripts
df_time['target_text'] = df_time.apply(lambda x: get_target_text(x['jsonurl'], x['starttime'], x['endtime']), axis=1)

In [ ]:
df_time.head()

In [ ]:
#save as CSV file
df_time.to_csv(path_or_buf = '../data/summary_with_target.csv', index=False)
df_cleaned = df_time[['filename','target_text','summary']]
df_cleaned.to_csv(path_or_buf = '../data/summary_with_target_cleaned.csv', index=False)

In [ ]:
df_cleaned.head(50)